In [3]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name}，帮我起一个具有{county}特色的{sex}名字")
prompts = prompt.format(name="算命大师", county="法国", sex="女孩")
print(prompts)

你是一个算命大师，帮我起一个具有法国特色的女孩名字


In [4]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个起名大师，你的名字叫"),
    ("human", "你好{name},你感觉如何？"),
    ("ai", "你好！我状态非常好!"),
    ("human", "你叫什么名字呢?"),
    ("ai", "你好！我叫{name}"),
    ("human", "{user_input}"),
])

chats = chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁？")
print(chats)

[SystemMessage(content='你是一个起名大师，你的名字叫', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好陈大师,你感觉如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！我状态非常好!', additional_kwargs={}, response_metadata={}), HumanMessage(content='你叫什么名字呢?', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！我叫陈大师', additional_kwargs={}, response_metadata={}), HumanMessage(content='你的爸爸是谁？', additional_kwargs={}, response_metadata={})]


In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate([
    ("system", "你是一个很厉害的人工智能助手"),
    MessagesPlaceholder("msgs")
])
result = prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})
print(result)

messages=[SystemMessage(content='你是一个很厉害的人工智能助手', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]


In [3]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

sy = SystemMessage(
    content = "你是一个起名大师",
    additional_kwargs = {"大师名字": "狗蛋"}
)
hu = HumanMessage(
    content="请问大师叫什么？"
)
ai = AIMessage(
    content="我叫你爹"
)
print([sy, hu, ai])

[SystemMessage(content='你是一个起名大师', additional_kwargs={'大师名字': '狗蛋'}, response_metadata={}), HumanMessage(content='请问大师叫什么？', additional_kwargs={}, response_metadata={}), AIMessage(content='我叫你爹', additional_kwargs={}, response_metadata={})]


In [11]:
##函数大师：根据函数名称，查找函数代码，并给出中文的代码说明
# Function Master: Given a function name, find the function code and provide a Chinese code explanation


from langchain_core.prompts import StringPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

# 定义一个简单的函数作为示例效果
# Define a simple function as an example
def hello_world(abc):
    print("Hello, world!")
    return abc


PROMPT = """\
你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: {function_name}
源代码:
{source_code}
代码解释:
"""

import inspect


def get_source_code(function_name):
    #获得源代码
    # Get the source code
    return inspect.getsource(function_name)

#自定义的模板class
# Custom template class
class CustmPrompt(StringPromptTemplate):


    def format(self, **kwargs) -> str:
        # 获得源代码
        # Get the source code
        source_code = get_source_code(kwargs["function_name"])

        # 生成提示词模板
        # Generate the prompt template
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

a = CustmPrompt(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

print(pm)

#和LLM连接起来
# Connect to LLM
from langchain_openai import OpenAI
import os
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)
msg = llm.invoke(pm)
print(msg)

你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
函数名称: hello_world
源代码:
def hello_world(abc):
    print("Hello, world!")
    return abc

代码解释:



BadRequestError: Error code: 400 - {'error': {'message': 'ip:port failed: json: cannot unmarshal array into Go struct field ip:port of type string (request id: 202510022118373120759637495192)', 'type': 'shell_api_error', 'param': '', 'code': None}}

In [17]:
# zeroshot会导致低质量回答
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
model="gpt-4o-mini",
temperature=0.0,
openai_api_base = os.getenv("OPENAI_API_BASE"),
openai_api_key = os.getenv("OPENAI_API_KEY")
)
res = model.invoke("What is 2 🦜 9?")
print("低质量回答")
print(res)

#增加示例
#使用FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
#增加示例组
examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
]
#构造提示词模板
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
#组合示例与提示词
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
#打印提示词模板
print(few_shot_prompt.invoke({}).to_messages())

## 最终提示词
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位神奇的数学奇才"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

##重新提问
chain = final_prompt | model
result = chain.invoke({"input": "What is 2 🦜 9?"})
print("使用few_shot后的高质量回答")
print(result)

低质量回答
content='The expression "2 🦜 9" seems to use a parrot emoji (🦜) in place of a mathematical operator. If you could clarify what operation you intend to represent with the parrot emoji, I would be happy to help you solve it!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 17, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CMYc7pECDPUAcwUFdS1hUE4FNimGP', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--d37d6503-d439-4661-96a5-221fd4088ad2-0' usage_metadata={'input_tokens': 17, 'output_tokens': 56, 'total_tokens': 73, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
[HumanMessag

In [1]:
# 安装向量数据库
! pip install chromadb==0.4.15 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
